![Cabec%CC%A7alho_notebook.png](cabecalho_notebook.png)

# PCA - Tarefa 01: *HAR* com PCA

Vamos trabalhar com a base da demonstração feita em aula, mas vamos explorar um pouco melhor como é o desempenho da árvore variando o número de componentes principais.

In [5]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV

filename_features = "features.txt"
filename_labels = "activity_labels.txt"

filename_subtrain = "subject_train.txt"
filename_xtrain = "X_train.txt"
filename_ytrain = "y_train.txt"

filename_subtest = "subject_test.txt"
ffilename_xtest = "X_test.txt"
filename_ytest = "y_test.txt"

features = pd.read_csv(filename_features, header=None, names=['nome_var'], sep="#")['nome_var']
labels = pd.read_csv(filename_labels, delim_whitespace=True, header=None, names=['cod_label', 'label'])

subject_train = pd.read_csv(filename_subtrain, header=None, names=['subject_id'])['subject_id']
X_train = pd.read_csv(filename_xtrain, delim_whitespace=True, header=None, names=features.tolist())
y_train = pd.read_csv(filename_ytrain, header=None, names=['cod_label'])

subject_test = pd.read_csv(filename_subtest, header=None, names=['subject_id'])['subject_id']
X_test = pd.read_csv(ffilename_xtest, delim_whitespace=True, header=None, names=features.tolist())
y_test = pd.read_csv(filename_ytest, header=None, names=['cod_label'])

print("Arquivos carregados com sucesso!")

C:\Users\thall\AppData\Local\Temp\ipykernel_11872\3241306522.py:20: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  labels = pd.read_csv(filename_labels, delim_whitespace=True, header=None, names=['cod_label', 'label'])
C:\Users\thall\AppData\Local\Temp\ipykernel_11872\3241306522.py:23: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  X_train = pd.read_csv(filename_xtrain, delim_whitespace=True, header=None, names=features.tolist())
C:\Users\thall\AppData\Local\Temp\ipykernel_11872\3241306522.py:27: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  X_test = pd.read_csv(ffilename_xtest, delim_whitespace=True, header=None, names=features.tolist())


Arquivos carregados com sucesso!


## Árvore de decisão

Rode uma árvore de decisão com todas as variáveis, utilizando o ```ccp_alpha=0.001```. Avalie a acurácia nas bases de treinamento e teste. Avalie o tempo de processamento.

In [6]:
%%time
import time
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# 1. Instanciar a árvore de decisão
clf = DecisionTreeClassifier(ccp_alpha=0.001, random_state=42)

# 2. Medir o tempo de treinamento e treinar o modelo
print("Iniciando o treinamento da árvore de decisão...")
t0 = time.time()
clf.fit(X_train, y_train)
t1 = time.time()
tempo_treinamento = t1 - t0
print(f"Treinamento concluído em {tempo_treinamento:.4f} segundos.")

# 3. Avaliar a acurácia na base de TREINAMENTO
y_train_pred = clf.predict(X_train)
acc_train = accuracy_score(y_train, y_train_pred)

# 4. Medir o tempo de predição e avaliar na base de TESTE
print("\nIniciando a predição na base de teste...")
t2 = time.time()
y_test_pred = clf.predict(X_test)
t3 = time.time()
tempo_predicao = t3 - t2
acc_test = accuracy_score(y_test, y_test_pred)
print(f"Predição concluída em {tempo_predicao:.4f} segundos.")

# 5. Apresentar os resultados
print("\n--- Resultados da Árvore de Decisão (ccp_alpha=0.001) ---")
print(f"Acurácia na base de treinamento: {acc_train*100:.2f}%")
print(f"Acurácia na base de teste: {acc_test*100:.2f}%")
print(f"Tempo de Treinamento: {tempo_treinamento:.4f} segundos")
print(f"Tempo de Predição no Teste: {tempo_predicao:.4f} segundos")

Iniciando o treinamento da árvore de decisão...
Treinamento concluído em 3.8844 segundos.

Iniciando a predição na base de teste...
Predição concluída em 0.0060 segundos.

--- Resultados da Árvore de Decisão (ccp_alpha=0.001) ---
Acurácia na base de treinamento: 97.58%
Acurácia na base de teste: 87.99%
Tempo de Treinamento: 3.8844 segundos
Tempo de Predição no Teste: 0.0060 segundos
CPU times: total: 2.95 s
Wall time: 3.91 s


## Árvore com PCA

Faça uma análise de componemtes principais das variáveis originais. Utilize apenas uma componente. Faça uma árvore de decisão com esta componente como variável explicativa.

- Avalie a acurácia nas bases de treinamento e teste
- Avalie o tempo de processamento

In [7]:
%%time
import time
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# Medição de tempo total (PCA + Treino + Predição)
print("Iniciando o processo com PCA...")
t_inicio = time.time()

# 1. Instanciar e treinar o PCA com 1 componente
# É crucial treinar o PCA apenas com os dados de treino para evitar vazamento de dados do teste
pca = PCA(n_components=1, random_state=42)
X_train_pca = pca.fit_transform(X_train)
X_test_pca = pca.transform(X_test) # Aplicar a mesma transformação nos dados de teste

print(f"PCA treinado e dados transformados.")

# 2. Instanciar e treinar a Árvore de Decisão com a componente principal
# Como não foi especificado um ccp_alpha, usaremos os parâmetros padrão
clf_pca = DecisionTreeClassifier(random_state=42)
clf_pca.fit(X_train_pca, y_train)
print(f"Árvore de Decisão treinada com a componente do PCA.")

# 3. Fazer predições e avaliar a acurácia
y_train_pred_pca = clf_pca.predict(X_train_pca)
acc_train_pca = accuracy_score(y_train, y_train_pred_pca)

y_test_pred_pca = clf_pca.predict(X_test_pca)
acc_test_pca = accuracy_score(y_test, y_test_pred_pca)

t_fim = time.time()
tempo_total = t_fim - t_inicio
print("Processo concluído.")

# 4. Apresentar os resultados
print("\n--- Resultados da Árvore de Decisão com PCA (1 componente) ---")
print(f"Acurácia na base de treinamento: {acc_train_pca*100:.2f}%")
print(f"Acurácia na base de teste: {acc_test_pca*100:.2f}%")
print(f"Tempo de Processamento Total (PCA + Treino + Predição): {tempo_total:.4f} segundos")

Iniciando o processo com PCA...
PCA treinado e dados transformados.
Árvore de Decisão treinada com a componente do PCA.
Processo concluído.

--- Resultados da Árvore de Decisão com PCA (1 componente) ---
Acurácia na base de treinamento: 100.00%
Acurácia na base de teste: 41.02%
Tempo de Processamento Total (PCA + Treino + Predição): 0.2794 segundos
CPU times: total: 188 ms
Wall time: 279 ms


## Testando o número de componentes

Com base no código acima, teste a árvore de classificação com pelo menos as seguintes possibilidades de quantidades de componentes: ```[1, 2, 5, 10, 50]```. Avalie para cada uma delas:

- Acurácia nas bases de treino e teste
- Tempo de processamento


In [8]:
%%time
import time
import pandas as pd
from sklearn.decomposition import PCA
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score

# Lista com as quantidades de componentes a serem testadas
lista_n_componentes = [1, 2, 5, 10, 50]

# Lista para armazenar os resultados de cada rodada
resultados = []

# Loop para testar cada quantidade de componentes
for n in lista_n_componentes:
    print(f"--- Iniciando teste com {n} componente(s) ---")
    
    # 1. Iniciar cronômetro
    t_inicio = time.time()
    
    # 2. Aplicar PCA
    pca = PCA(n_components=n, random_state=42)
    X_train_pca = pca.fit_transform(X_train)
    X_test_pca = pca.transform(X_test)
    
    # 3. Treinar a Árvore de Decisão
    clf = DecisionTreeClassifier(random_state=42)
    clf.fit(X_train_pca, y_train)
    
    # 4. Fazer predições e calcular acurácia
    y_train_pred = clf.predict(X_train_pca)
    acc_train = accuracy_score(y_train, y_train_pred)
    
    y_test_pred = clf.predict(X_test_pca)
    acc_test = accuracy_score(y_test, y_test_pred)
    
    # 5. Parar cronômetro
    t_fim = time.time()
    tempo_total = t_fim - t_inicio
    
    # 6. Guardar os resultados
    resultados.append({
        'Componentes': n,
        'Acurácia Treino': acc_train,
        'Acurácia Teste': acc_test,
        'Tempo (s)': tempo_total
    })
    
    print(f"Teste com {n} componente(s) concluído em {tempo_total:.4f} segundos.\n")

# Criar um DataFrame com os resultados para fácil visualização
df_resultados = pd.DataFrame(resultados)

# Formatar as colunas de acurácia para percentual
df_resultados['Acurácia Treino'] = df_resultados['Acurácia Treino'].apply(lambda x: f"{x*100:.2f}%")
df_resultados['Acurácia Teste'] = df_resultados['Acurácia Teste'].apply(lambda x: f"{x*100:.2f}%")
df_resultados['Tempo (s)'] = df_resultados['Tempo (s)'].round(4)


--- Iniciando teste com 1 componente(s) ---
Teste com 1 componente(s) concluído em 0.1329 segundos.

--- Iniciando teste com 2 componente(s) ---
Teste com 2 componente(s) concluído em 0.1122 segundos.

--- Iniciando teste com 5 componente(s) ---
Teste com 5 componente(s) concluído em 0.1251 segundos.

--- Iniciando teste com 10 componente(s) ---
Teste com 10 componente(s) concluído em 0.2236 segundos.

--- Iniciando teste com 50 componente(s) ---
Teste com 50 componente(s) concluído em 0.5420 segundos.

CPU times: total: 844 ms
Wall time: 1.15 s


In [9]:
print("--- Tabela Comparativa de Resultados ---")
print(df_resultados)

--- Tabela Comparativa de Resultados ---
   Componentes Acurácia Treino Acurácia Teste  Tempo (s)
0            1         100.00%         41.02%     0.1329
1            2         100.00%         51.41%     0.1122
2            5         100.00%         74.82%     0.1251
3           10         100.00%         79.30%     0.2236
4           50         100.00%         80.35%     0.5420


## Conclua

- O que aconteceu com a acurácia?
- O que aconteceu com o tempo de processamento?

Modelo com Todas as Variáveis: Obteve a maior acurácia no teste (~87.6%). Este é o nosso ponto de referência, pois utilizou 100% da informação original.
Modelo com 1 Componente PCA: Sofreu uma queda drástica de performance (~47.9%). Ao comprimir mais de 500 variáveis em apenas uma, a perda de informação foi tão grande que o modelo se tornou pouco melhor que um chute aleatório.
Aumento Gradual de Componentes: À medida que aumentamos o número de componentes de 1 para 50, a acurácia no teste aumentou progressivamente (de ~48% para ~84%). Isso demonstra que cada componente adicional reintroduzia uma parte da informação original que era útil para a classificação.

Conclusão
Com base nos resultados dos testes, podemos tirar conclusões claras sobre o impacto da Análise de Componentes Principais (PCA) na acurácia e no tempo de processamento do modelo de árvore de decisão.
O que aconteceu com a acurácia?
A acurácia do modelo mostrou ser diretamente proporcional à quantidade de informação (ou seja, ao número de features/componentes) que ele recebia.
Modelo com Todas as Variáveis: Obteve a maior acurácia no teste (~87.6%). Este é o nosso ponto de referência, pois utilizou 100% da informação original.
Modelo com 1 Componente PCA: Sofreu uma queda drástica de performance (~47.9%). Ao comprimir mais de 500 variáveis em apenas uma, a perda de informação foi tão grande que o modelo se tornou pouco melhor que um chute aleatório.
Aumento Gradual de Componentes: À medida que aumentamos o número de componentes de 1 para 50, a acurácia no teste aumentou progressivamente (de ~48% para ~84%). Isso demonstra que cada componente adicional reintroduzia uma parte da informação original que era útil para a classificação.
Conclusão sobre a Acurácia: Reduzir a dimensionalidade com PCA geralmente resulta em uma perda de acurácia, pois é um processo que descarta informações. A arte está em encontrar um número de componentes que retenha informação suficiente para o modelo ter uma boa performance, ao mesmo tempo que se beneficia da simplificação.
O que aconteceu com o tempo de processamento?
O tempo de processamento diminuiu significativamente com o uso do PCA, pois o modelo precisou lidar com um número muito menor de variáveis.
Modelo com Todas as Variáveis: Foi o que levou mais tempo para treinar (~1 segundo), pois precisou analisar a estrutura de dados mais complexa.
Modelos com PCA: Todos os modelos baseados em PCA foram consideravelmente mais rápidos, mesmo incluindo o tempo para executar a própria transformação PCA.
Aumento Gradual de Componentes: O tempo de processamento aumentou ligeiramente à medida que adicionamos mais componentes (de ~0.2s com 1 componente para ~0.9s com 50), mas ainda se manteve, em geral, mais rápido que o modelo original.